# Neural Networks
*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. Please check the pdf file for more details.*

In this exercise you will:
    
- implement the **forward** and **backward** operations for different layers in neural networks
- implement a simple neural networks for classification

Please note that **YOU CANNOT USE ANY MACHINE LEARNING PACKAGE SUCH AS SKLEARN** for any homework, unless you are asked.

In [1]:
# some basic imports
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import math

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
# %load gradient_check.py
import scipy.io as sio
import numpy as np
from feedforward_backprop import feedforward_backprop
np.random.seed(1022)
digit_data = sio.loadmat('digit_data.mat')
X = digit_data['X']
y = digit_data['y']
_, num_cases = X.shape
train_num_cases = num_cases * 4 // 5
X = X.reshape((400, num_cases))
X = X.reshape((num_cases, 400))
# X has the shape of (number of samples, number of pixels)
train_data = X[:train_num_cases,:]
train_label = y[:, :train_num_cases]
test_data = X[train_num_cases:num_cases, :]
test_label = y[:,train_num_cases:]
weights = {}
weights['fully1_weight'] = np.random.randn(400, 25) / 400
weights['fully1_bias'] = np.random.rand(25, 1) 
weights['fully2_weight'] = np.random.randn(25, 10) / 25
weights['fully2_bias'] = np.random.rand(10, 1)

data = train_data[:100, :]
label = train_label[:, :100]

EPSILON = 0.00010
loss, _, grads = feedforward_backprop(data, label, weights)

# check correctness of fully1_bias's gradient
for c in range(weights['fully1_bias'].shape[0]):
    weights['fully1_bias'][c, 0] = weights['fully1_bias'][c, 0] + EPSILON
    loss_2, _, grads_2 = feedforward_backprop(data, label, weights)
    print('{:.2}, {:.2}, {:.2}'.format((loss_2 - loss) / EPSILON, 
                                       grads['fully1_bias_grad'][c, 0], 
                                       grads_2['fully1_bias_grad'][c, 0]))
    weights['fully1_bias'][c, 0] = weights['fully1_bias'][c, 0] - EPSILON



0.0015, 0.0015, 0.0015
0.003, 0.003, 0.003
-0.0012, -0.0012, -0.0012
0.0058, 0.0058, 0.0058
-0.01, -0.01, -0.01
-0.0041, -0.0041, -0.0041
-0.0044, -0.0044, -0.0044
-0.0017, -0.0017, -0.0017
0.0033, 0.0033, 0.0033
0.0058, 0.0058, 0.0058
-0.0025, -0.0025, -0.0025
0.0016, 0.0016, 0.0016
0.0024, 0.0024, 0.0024
-0.0041, -0.0041, -0.0041
-0.012, -0.012, -0.012
-0.0021, -0.0021, -0.0021
0.00039, 0.00039, 0.00039
0.0072, 0.0072, 0.0072
0.0054, 0.0054, 0.0054
-0.005, -0.005, -0.005
0.0081, 0.0081, 0.0081
0.00099, 0.00099, 0.00099
0.0085, 0.0085, 0.0085
0.0051, 0.0051, 0.0051
0.0015, 0.0015, 0.0015


## Training

In [3]:
digit_data = sio.loadmat('digit_data.mat')
X = digit_data['X']
y = digit_data['y']
_, num_cases = X.shape
train_num_cases = num_cases * 4 // 5
X = X.reshape((400, num_cases))
X = X.transpose()
# X has the shape of (number of samples, number of pixels)
train_data = X[:train_num_cases,:]
train_label = y[:, :train_num_cases]
test_data = X[train_num_cases:, :]
test_label = y[:, train_num_cases:]
weights = {}
weights['fully1_weight'] = np.random.randn(400, 25) / 400
weights['fully1_bias'] = np.random.rand(25, 1) 
weights['fully2_weight'] = np.random.randn(25, 10) / 25
weights['fully2_bias'] = np.random.rand(10, 1)

In [4]:
# training setting
weight_inc = {}
for name in ('fully1_weight', 'fully1_bias', 'fully2_weight', 'fully2_bias'):
    weight_inc[name] = np.zeros(weights[name].shape)
batch_size = 100
max_epoch = 10
momW = 0.9
wc = 0.0005
learning_rate = 0.1

# Training iterations
from get_new_weight_inc import get_new_weight_inc
from feedforward_backprop import feedforward_backprop

for epoch in range(max_epoch):
    for i in range(math.ceil(train_num_cases/batch_size)):
        data = train_data[i * batch_size:min((i + 1) * batch_size, train_num_cases), :]
        label = train_label[:, i * batch_size:min((i + 1) * batch_size, train_num_cases)]
        # The feedforward and backpropgation processes
        loss, accuracy, gradients = feedforward_backprop(data, label, weights)
        print('{:3}.{:2} loss:{:.3}, accuracy:{}'.format(epoch + 1, i + 1, loss, accuracy))
        # Updating weights
        for name in ('fully1_weight', 'fully1_bias', 'fully2_weight', 'fully2_bias'):
            weight_inc[name] = get_new_weight_inc(weight_inc[name], weights[name], momW, wc, learning_rate, gradients[name + '_grad'])
            weights[name] += weight_inc[name]
        

  1. 1 loss:2.31, accuracy:0.11
  1. 2 loss:2.33, accuracy:0.11
  1. 3 loss:2.29, accuracy:0.11
  1. 4 loss:2.32, accuracy:0.1
  1. 5 loss:2.28, accuracy:0.11
  1. 6 loss:2.28, accuracy:0.1
  1. 7 loss:2.28, accuracy:0.08
  1. 8 loss:2.26, accuracy:0.35
  1. 9 loss:2.24, accuracy:0.27
  1.10 loss:2.23, accuracy:0.33
  1.11 loss:2.19, accuracy:0.35
  1.12 loss:2.15, accuracy:0.29
  1.13 loss:2.14, accuracy:0.29
  1.14 loss:2.12, accuracy:0.28
  1.15 loss:2.01, accuracy:0.39
  1.16 loss:1.92, accuracy:0.46
  1.17 loss:1.87, accuracy:0.46
  1.18 loss:1.79, accuracy:0.52
  1.19 loss:1.77, accuracy:0.44
  1.20 loss:1.63, accuracy:0.64
  1.21 loss:1.49, accuracy:0.59
  1.22 loss:1.43, accuracy:0.58
  1.23 loss:1.26, accuracy:0.68
  1.24 loss:1.12, accuracy:0.75
  1.25 loss:1.07, accuracy:0.7
  1.26 loss:0.994, accuracy:0.74
  1.27 loss:1.08, accuracy:0.63
  1.28 loss:0.896, accuracy:0.8
  1.29 loss:0.842, accuracy:0.85
  1.30 loss:0.829, accuracy:0.76
  1.31 loss:0.789, accuracy:0.76
  1.32 

  7.13 loss:0.205, accuracy:0.94
  7.14 loss:0.286, accuracy:0.92
  7.15 loss:0.0857, accuracy:0.97
  7.16 loss:0.161, accuracy:0.97
  7.17 loss:0.154, accuracy:0.95
  7.18 loss:0.234, accuracy:0.94
  7.19 loss:0.28, accuracy:0.9
  7.20 loss:0.162, accuracy:0.93
  7.21 loss:0.188, accuracy:0.94
  7.22 loss:0.168, accuracy:0.94
  7.23 loss:0.17, accuracy:0.97
  7.24 loss:0.132, accuracy:0.95
  7.25 loss:0.102, accuracy:0.98
  7.26 loss:0.132, accuracy:0.96
  7.27 loss:0.255, accuracy:0.93
  7.28 loss:0.175, accuracy:0.96
  7.29 loss:0.224, accuracy:0.93
  7.30 loss:0.169, accuracy:0.96
  7.31 loss:0.177, accuracy:0.95
  7.32 loss:0.137, accuracy:0.96
  7.33 loss:0.196, accuracy:0.95
  7.34 loss:0.215, accuracy:0.94
  7.35 loss:0.301, accuracy:0.92
  7.36 loss:0.172, accuracy:0.94
  7.37 loss:0.147, accuracy:0.94
  7.38 loss:0.173, accuracy:0.95
  7.39 loss:0.173, accuracy:0.96
  7.40 loss:0.277, accuracy:0.93
  8. 1 loss:0.126, accuracy:0.95
  8. 2 loss:0.159, accuracy:0.95
  8. 3 loss:

## Testing

In [5]:
# TODO
loss, accuracy, _ = feedforward_backprop(test_data, test_label, weights)
print('loss:{:.3}, accuracy:{}'.format(loss, accuracy))

loss:0.246, accuracy:0.924
